In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from utils.test_env import EnvTest


## Human Control

In [3]:
env = EnvTest((5, 5, 1))


In [42]:
done = False

obs = env.reset()

while not done:
    
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
    env.render()
    print("Reward: ", reward)
    
    
    

3
Reward:  -0.1
2
Reward:  0.0
0
Reward:  -1.0
3
Reward:  -0.1
3
Reward:  -0.1


## Linear Q Learning

In [43]:
from tf2_linear import Linear
import datetime
import tensorflow as tf

In [44]:
episodes = 50000

In [45]:
env = EnvTest((5, 5, 1))


In [ ]:
# def run(env, policy, model, verbose=0):
#     done = False
#     curr_state = env.reset()
#     total_reward = 0
#     steps = 0
#     total_loss = 0
    
#     while not done:
#         action, action_values = policy(curr_state)
#         next_state, reward, done, info = env.step(action)
#         loss = model.update(curr_state, action, reward, next_state, done)
#         total_reward += reward
#         total_loss += loss
#         steps += 1
        
#         if verbose > 0:
#             print("State: {}".format(np.sum(curr_state)))
#             print("Action Values: {} Action: {}".format(action_values, action))
#             print("Loss: {}".format(loss))
#             print("Reward: {}".format(reward))
#             print("")

#         curr_state = next_state
        
#     return total_reward, total_loss, steps

In [46]:
def replay_run(env, policy, model, verbose=0):
    done = False
    curr_state = env.reset()
    total_reward = 0
    steps = 0
    total_loss = 0
    
    while not done:
        action, action_values = policy(curr_state)
        next_state, reward, done, info = env.step(action)
    
        model.replay_fill(curr_state, action, reward, next_state, done)
        loss = model.replay_update()
        
        total_reward += reward
        total_loss += loss
        steps += 1
        
        if verbose > 0:
            print("State: {}".format(np.sum(curr_state)))
            print("Action Values: {} Action: {}".format(action_values, action))
            print("Target Values: {}".format(model.target_net(model.preprocess(next_state))))
            print("Loss: {}".format(loss))
            print("Reward: {}".format(reward))
            print("")

        curr_state = next_state
        
    return total_reward, total_loss/steps, steps

In [47]:
buffer_len = 500
model = Linear(replay_buffer_len=buffer_len)

# Clear any logs from previous runs
# !rm -rf ./logs

# log_dir = "logs/experiment/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "logs/debugging"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
Writer = tf.summary.create_file_writer(log_dir)

update_steps = 0
run_episodes = 0

In [ ]:
# Fill Replay Buffer

while model.replay_buffer.num_in_buffer < buffer_len:
    done = False
    curr_state = env.reset()

    while not done:
        action = np.random.randint(5)
        next_state, reward, done, info = env.step(action)
        model.replay_fill(curr_state, action, reward, next_state, done)
        curr_state = next_state

print(model.replay_buffer.num_in_buffer)  

In [ ]:
# #Test Overfit
# for i in range(episodes):
#     update_steps += 1
#     loss = model.replay_update()
#     if i % 2 == 0:
#         with Writer.as_default():
#             tf.summary.scalar('Loss', loss,
#                               step=update_steps)



In [ ]:
# Random Run to get State to Track
done = False
curr_state = env.reset()
track_states = [curr_state]

while not done:
    
    action = np.random.randint(5)
    next_state, reward, done, info = env.step(action)
    
    popped = False
    for s in track_states:
        if np.all(s==next_state):
            popped = True
            
    if not popped:
        track_states.append(next_state)
       
print(len(track_states))


In [ ]:
for i in range(episodes):
    total_reward, train_loss, steps,  = replay_run(env, model.policy, model)
    update_steps += steps
    run_episodes += 1
    
    if i % 10 == 0:        
        with Writer.as_default():
            tf.summary.scalar('Avg Loss', train_loss,
                              step=update_steps)
                              
    if i % 50 == 0:
        total_test_reward, test_loss, steps = replay_run(env, model.greedy_policy, model)
        update_steps += steps

        action, action_values = model.greedy_policy(track_states[0])
        q_state_0_a0 = action_values[0,0] 
        q_state_0_a1 = action_values[0,1] 
        q_state_0_a2 = action_values[0,2] 
        q_state_0_a3 = action_values[0,3] 
        q_state_0_a4 = action_values[0,4] 
        
        with Writer.as_default():

            tf.summary.scalar('Epsiode Reward', total_test_reward,
                              step=update_steps)

            tf.summary.scalar('Q(s0,a0)', q_state_0_a0, step=update_steps)            
            tf.summary.scalar('Q(s0,a1)', q_state_0_a1, step=update_steps)  
            tf.summary.scalar('Q(s0,a2)', q_state_0_a2, step=update_steps)  
            tf.summary.scalar('Q(s0,a3)', q_state_0_a3, step=update_steps)  
            tf.summary.scalar('Q(s0,a4)', q_state_0_a4, step=update_steps)  



In [ ]:
total_test_reward, test_loss, steps = replay_run(env, model.greedy_policy, model, verbose=1)
